# Counting Stability Chambers of Moduli Spaces

This notebook shows how this partitioning algorithm is useful in **multiple areas of interest**.
In particular, it has been used in **state-of-the-art research on moduli spaces of parabolic vector bundles**,
where it allowed us to **count the number of stability chambers** of these moduli spaces with parameters $n$ (number of stability points) and $r$ (rank of vector bundles).

For background see:

> [_A computational analysis of isomorphism classes of moduli spaces of parabolic vector bundles_](https://repositorio.comillas.edu/xmlui/handle/11531/89959)


## Step 1. Imports and Setup


In [ ]:
import json
from time import perf_counter


from polypart.ppart import build_partition_tree
from polypart.polytopes import get_simplex
from polypart.arrangements import get_moduli_arrangement
from polypart.io import save_tree

## Step 2. Build the Polytope

In a moduli space of parabolic vector bundles, the ambient space is a product of simplices of dimension $n(r-1)$.
We can build this polytope using the predefined function `polypart.polytopes.get_simplex`.

We will study the case of $n=1$ and $r=7$, which gives a simplex of dimension $6$ with $7$ facets.


In [2]:
n, r = 1, 7
simplex = get_simplex(r - 1)
simplex.extreme()
simplex

Polytope(dim=6, n_ineq=7, n_vertices=7)

## Step 3. Generate Hyperplane Arrangement

In the moduli space classification problem, there are some hyperplanes called "stability walls" that partition the parameter space (starting polytope) into chambers, where each chamber represents a different moduli space.

We will generate these hyperplanes using the predefined function `polypart.arrangements.get_moduli_arrangement`, which implements the combinatorial construction of these hyperplanes for the given parameters $n$, $r$, and $d$ (degree of the vector bundles, which we set to $0$ for simplicity).


In [3]:
hyperplanes = get_moduli_arrangement(n, r, d=0)
print(f"Generated arrangement with {len(hyperplanes)} hyperplanes.")

Generated arrangement with 65 hyperplanes.


## Step 4. Build the Partition Tree

We now apply the partitioning algorithm to the polytope and the set of hyperplanes using to different strategies: 'random' and 'v-entropy'. We compare the runtime and the maximum and average depth of the resulting trees.


- 'Random' strategy: selects hyperplanes randomly at each node.


In [4]:
start = perf_counter()
tree, n_chambers = build_partition_tree(simplex, hyperplanes, strategy="random")
elapsed = perf_counter() - start
print(f"Found {n_chambers} chambers in {elapsed:.2f} s")


print("Partition tree statistics:")
print(json.dumps(tree.stats(include_per_depth_stats=False), indent=4))

Found 1296 chambers in 1.32 s
Partition tree statistics:
{
    "total_nodes": 2591,
    "avg_depth": 14.483796296296296,
    "max_depth": 22,
    "avg_candidates": 2.605557699729834,
    "avg_inequalities": 8.060980316480123,
    "avg_vertices": 9.067927441142416
}


- 'V-entropy' strategy: maximizes volume entropy (approximated with number of vertices in each sub-polytope).


In [7]:
start = perf_counter()
tree, n_chambers = build_partition_tree(
    simplex, hyperplanes, strategy="v-entropy", remove_redundancies=True
)
elapsed = perf_counter() - start
print(f"Found {n_chambers} chambers in {elapsed:.2f} s")

print(json.dumps(tree.stats(include_per_depth_stats=False), indent=4))

Found 1296 chambers in 1.29 s
{
    "total_nodes": 2591,
    "avg_depth": 11.247685185185185,
    "max_depth": 14,
    "avg_candidates": 2.074488614434581,
    "avg_inequalities": 7.947510613662678,
    "avg_vertices": 8.631416441528367
}


## Step 6. Saving the Tree (Optional)

The decision tree is very useful in research for identifying **isomorphisms** between stability chambers, since it allows for efficient point location queries, and it also allows for a detailed analysis of the structure of the partition.


In [6]:
# save_tree(tree, f"moduli_n{n}_r{r}_v_entropy.json")
